Load deps

In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import DutchStemmer
from collections import defaultdict, Counter
import nltk
import json
from math import log10, sqrt
nltk.download('punkt')
stemmer = DutchStemmer()

[nltk_data] Downloading package punkt to /home/ffk27/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


De complete bijbel in het data-object stoppen met de volgende datastructuur:
```
{
    translation: 'NBG 1951',
    language: 'DUT',
    id: 'NBG51'
    books: [{
        bname: 'Genesis',
        bsname: 'Gen',
        chapters:[
            ['In den beginne', 'Vers 2'],   # Hoofdstuk 1
            ['Vers 1', 'Vers 2']            # Hoofdstuk 2
        ]
    }]
}
```

In [4]:
filename = 'NBV.json'

data = {}

with open(filename) as json_file:
    data = json.load(json_file)
    print('Vertaling:', data['translation'])

def getVers(pos):
    return data['books'][pos[0]-1]['chapters'][pos[1]-1][pos[2]-1]

def posName(pos):
    return data['books'][pos[0]-1]['bname'] + ' ' + str(pos[1]) + ':' + str(pos[2])

pos = (1, 1, 1)

print(posName(pos), ':')
getVers(pos)

Vertaling: De Nieuwe Bijbelvertaling
Genesis 1:1 :


'In het begin schiep God de hemel en de aarde. '

Een inverted index opbouwen van ieder woord van elk vers. De woorden worden m.b.v. de stem-functie geminimaliseerd. Aan ieder woord wordt de positie van het vers toegevoegd. De positie heeft is een tupel met (boeknr, hoofdstuknr, versnr)
Voorbeeld: ```inverted_index['In'] = [(1,1,1)...enz]```

In [5]:
inverted_index = defaultdict(list)
tf_matrix = defaultdict(Counter)

def preprocess(tokens):
    for i, token in enumerate(tokens):
        tokens[i] = stemmer.stem(token)
    return tokens

def buildIndeces():
    for i, book in enumerate(data['books']):
        for j, chapter in enumerate(book['chapters']):
            tf_matrix[(i+1,j+1)] = Counter()
            for k, vers in enumerate(chapter):
                tokens = preprocess(word_tokenize(vers))
                tf_matrix[(i+1,j+1)] += Counter(tokens)
                for word in tokens:
                    inverted_index[word].append((i+1, j+1, k+1))

buildIndeces()
print(len(inverted_index), 'verschillende woorden geindexeerd.')
tf_matrix[(1,1)].most_common(10) # woordtellingen Genesis 1

21450 verschillende woorden geindexeerd.


[('de', 56),
 ('het', 52),
 ('en', 51),
 ('.', 45),
 (',', 29),
 ('god', 24),
 (':', 15),
 ('aard', 14),
 ('van', 13),
 ('over', 12)]

Functie's voor tf-idf:

In [6]:
def idf(t):
    return (num_documents + 1) / (df(t) + 1)

def tfidf(t,d): # returns tf-idf score of term t in document d
    return tf(t,d) * idf(t)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(tf_matrix))

print('Aantal hoofdstukken:', num_documents)
print('Aantal keer het woord god in Genesis 1:', tf('god', (1,1)))
print('Aantal keer het woord god in de hele bijbel:', df('god'))
print('Inverse document freqency:', idf('god'))
print('Tf-idf:', tfidf('god', (1,1)))

Aantal hoofdstukken: 1346.0
Aantal keer het woord god in Genesis 1: 24.0
Aantal keer het woord god in de hele bijbel: 5020.0
Inverse document freqency: 0.2682732523401713
Tf-idf: 6.438558056164111


Zoeken op 1 woord

In [7]:
results = [(pos, tfidf('god', (pos[0], pos[1]))) for pos in inverted_index['god']]
resultsc = []
chapter = (0, 0)
for res in sorted(results, key=lambda tup: tup[1], reverse=True):
    ch = res[0][0:2]
    if (chapter ==  ch):
        resultsc[len(resultsc)-1].append(res[0])
    else:
        resultsc.append([res[0]])
        chapter = ch
for res in resultsc[0:10]:
    print(posName(res[0]), '\t-',getVers(res[0]))

Brief van Jeremia 1:1 	- Omdat jullie gezondigd hebben tegen God, zullen jullie in ballingschap naar Babylonië worden weggevoerd door Nebukadnessar, de koning van de Babyloniërs.
Psalmen 68:1 	- Voor de koorleider. Van David, een psalm, een lied. (68:2) God staat op, zijn vijanden stuiven uiteen, zijn haters vluchten als hij verschijnt. 
1 Johannes 4:1 	- Geliefde broeders en zusters, vertrouw niet elke geest. Onderzoek altijd of een geest van God komt, want er zijn veel valse profeten in de wereld verschenen. 
Deuteronomium 4:1 	- Luister dus, Israël, naar de wetten en de regels waarin ik u onderwijs en kom ze na. Dan blijft u in leven en kunt u het land in bezit nemen dat de HEER, de God van uw voorouders, u zal geven. 
Genesis 1:1 	- In het begin schiep God de hemel en de aarde. 
Romeinen 1:1 	- Van Paulus, dienaar van Christus Jezus, geroepen tot apostel en uitgekozen om het evangelie van God te verkondigen, 
Romeinen 8:3 	- Waartoe de wet niet in staat was, machteloos als hij was 

In [33]:

def findOccurrences(term):
    for s in [key for key in inverted_index.keys() if term in key]:
        for pos in inverted_index[s]:
            print(posName(pos), getVers(pos), '\n')

findOccurrences('stenig')


Exodus 8:26 (8:22) 'Dat is onmogelijk, 'zei Mozes. 'De offers die wij de HEER, onze God, moeten brengen, zullen de Egyptenaren weerzinwekkend vinden. Als we in hun bijzijn dergelijke offers brengen, stenigen ze ons nog!  

Exodus 17:4 Mozes riep luid de HEER aan. 'Wat moet ik met dit volk beginnen?' vroeg hij. 'Er hoeft niet veel meer te gebeuren of ze stenigen mij!'  

Leviticus 20:2 'Zeg tegen de Israëlieten: "Wanneer een Israëliet of een vreemdeling die in Israël woont een van zijn kinderen aan Moloch offert, moet hij ter dood gebracht worden; het volk moet hem stenigen.  

Numeri 14:10 Het volk dreigde hen te stenigen, maar toen verscheen de majesteit van de HEER in de ontmoetingstent aan de Israëlieten.  

Numeri 15:34 De HEER zei tegen Mozes: 'Die man moet gedood worden. De hele gemeenschap moet hem buiten het kamp stenigen.'  

Numeri 15:35 Toen brachten ze hem met zijn allen buiten het kamp, en daar doodden ze hem door hem te stenigen, zoals de HEER Mozes had opgedragen.  

Deu

In [8]:
def or_query(tokens):
    indexes = set()
    for token in tokens:
        pos = inverted_index[token]
        indexes.update((pos[0], pos[1]))
    return indexes

def score_ntn(query_words, doc_id):
    score = 0.0
    for t in query_words:
        score += tfidf(t, doc_id)
    return score

def query_ntn(query_string):
    docs = defaultdict(list)
    tokenized = preprocess(word_tokenize(query_string))
    for doc_id in or_query(tokenized):
        print(doc_id)
        docs[doc_id] = score_ntn(tokenized, doc_id)
    return Counter(docs).most_common(10)
    
print(query_ntn('god'))

(1, 1, 1)
(1, 1, 2)
[((1, 1, 1), 0.0), ((1, 1, 2), 0.0)]
